In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import csv
import os
import numpy as np
import random
tf.enable_eager_execution()
AUTOTUNE = tf.data.experimental.AUTOTUNE # Sets number of elements to prefetch

In [ ]:
'''
EfficientNetB0 - (224, 224, 3)
EfficientNetB1 - (240, 240, 3)
EfficientNetB2 - (260, 260, 3)
EfficientNetB3 - (300, 300, 3)

EfficientNetB4 - (380, 380, 3)
EfficientNetB5 - (456, 456, 3)
EfficientNetB6 - (528, 528, 3)
EfficientNetB7 - (600, 600, 3)
'''

%cd ../input/efficientnet-repo
import efficientnet.efficientnet.tfkeras as efn
%cd ../../working

input_shape = (256 , 256 , 3)
weights_path = '../input/efficientnet-keras-weights-b0b5/efficientnet-b3_imagenet_1000_notop.h5'
efn_model = efn.EfficientNetB3(weights=weights_path, include_top=False, input_shape=input_shape)
print('Model imported.')

In [ ]:
# fine_tune_at = 523 # Start training on block 7 (see model.summary() for layer info)
# for layer in model.layers[:fine_tune_at]:
#     layer.trainable =  False
# model.summary()
efn_model.trainable = False
def build_model():
    l2 = 0.00001
    model = tf.keras.models.Sequential()
    model.add(efn_model)
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dropout(rate=0.5))
    model.add(tf.keras.layers.Dense(1200, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2)))
    model.add(tf.keras.layers.Dropout(rate=0.5))
    model.add(tf.keras.layers.Dense(500, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2)))
    model.add(tf.keras.layers.Dropout(rate=0.5))
    model.add(tf.keras.layers.Dense(100, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2)))
    model.add(tf.keras.layers.Dropout(rate=0.5))
    model.add(tf.keras.layers.Dense(5, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(l2)))
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        optimizer = tf.keras.optimizers.Adam(lr=0.0005),
        metrics=['accuracy']
    )
    
    return model

model = build_model()
print('Model Built')

In [ ]:

# '''
# Reads entire preprocessed data into RAM.
# '''
# def load_data(image_folder_path, label_path):  
#     '''
#     Loading training and testing databases. 
#     '''
    
#     # Reading in labels and creating a dictionary
#     reader = csv.reader(open(label_path, 'r')) # Reads in header as well but thats not important. 
#     label_dict = dict([(key, value) for key, value in reader])
        
#     image_tensors = []
#     image_labels = []
#     for name in os.listdir(image_folder_path):    
#         image_path = os.path.join(image_folder_path, name) 
#         image_file = tf.read_file(image_path)
#         image_tensor = tf.image.decode_png(image_file)
#         image_tensors.append(image_tensor)
        
#         no_extension = name[:name.index('.')]
#         image_labels.append(int(label_dict[no_extension]))
        
#         if len(image_tensors) == 2000:
#             break
    
#     # Normalize image input. 
#     image_ds = tf.data.Dataset.from_tensor_slices(image_tensors)
#     def normalize(image):
#         return tf.cast(image/255,tf.float32) 
#     image_ds = image_ds.map(normalize, num_parallel_calls=AUTOTUNE)
    
#     label_ds = tf.data.Dataset.from_tensor_slices(image_labels)
#     ds = tf.data.Dataset.zip((image_ds, label_ds))
       
#     # Now determining how data is fed.
#     ds = ds.shuffle(buffer_size=len(image_tensors)).repeat().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

#     return ds, len(image_tensors)
    
# train_image_ds, train_ds_size = load_data('/kaggle/input/preprocessed/Preprocessed_Data', '/kaggle/input/aptos2019-blindness-detection/train.csv')
# #test_image_ds = load_data('/kaggle/input/aptos2019-blindness-detection/test_images', '/kaggle/input/aptos2019-blindness-detection/test.csv')    


In [ ]:
IMAGE_FOLDER_PATH = '/kaggle/input/preprocessed-aug/Preprocessed_Data'
LABEL_PATH = '/kaggle/input/aptos2019-blindness-detection/train.csv'
TRAIN_BATCH_SIZE = 32
VAL_BATCH_SIZE = 50
EPOCHS = 50
PERCENT_TRAIN = 0.8

def load_data(file_names, label_dict):
    batch_shape = (len(file_names),) + input_shape
    images = np.zeros(batch_shape)
    labels = np.zeros(len(file_names))
    i = 0
    for file_name in file_names:    
        image_path = os.path.join(IMAGE_FOLDER_PATH, file_name) 
        image_file = tf.read_file(image_path)
        image_tensor = tf.image.decode_png(image_file)/255 # Normalize
        images[i, ...] = image_tensor.numpy()
        
        no_extension = file_name[:file_name.index('.')]
        if no_extension[0] == '~':
            removed_tilda = no_extension[1:]
            labels[i] = int(label_dict[removed_tilda])
        else:
            labels[i] = int(label_dict[no_extension])
        
        i = i + 1
    
    return images, labels

def image_loader(file_names, label_dict, batch_size):
    L = len(file_names)
    #this line is just to make the generator infinite, keras needs that    
    while True:
        random.shuffle(file_names)
        batch_start = 0
        batch_end = batch_size
        while batch_start < L:
            limit = min(batch_end, L)
            images, labels = load_data(file_names[batch_start:limit], label_dict)

            yield (images, labels) #a tuple with two numpy arrays with batch_size samples     

            batch_start += batch_size   
            batch_end += batch_size
            
# Reading in labels and creating a dictionary
reader = csv.reader(open(LABEL_PATH, 'r')) # Reads in header as well but thats not important. 
label_dict = dict([(key, value) for key, value in reader])

file_names = os.listdir(IMAGE_FOLDER_PATH)
random.shuffle(file_names)
split_index = int(np.ceil(len(file_names)*PERCENT_TRAIN))
train_file_names = file_names[:split_index]
val_file_names = file_names[split_index:]

print('Data ready')

In [ ]:
'''
Train model
'''
callbacks = [
  # Interrupt training if `val_loss` stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=8, monitor='val_loss'),
  tf.keras.callbacks.ModelCheckpoint('model_weights', verbose=1, save_weights_only=True,period=3)
]

steps_per_epoch = np.ceil(
    float(
        len(train_file_names)
    )/TRAIN_BATCH_SIZE
)
validation_steps = np.ceil(
    float(
        len(val_file_names)
    )/VAL_BATCH_SIZE
)
model.fit_generator(image_loader(train_file_names, label_dict, TRAIN_BATCH_SIZE), 
                    epochs=EPOCHS, 
                    steps_per_epoch=steps_per_epoch, 
                    callbacks=callbacks,
                    validation_data=image_loader(val_file_names, label_dict, VAL_BATCH_SIZE),
                    validation_steps=validation_steps)

In [ ]:
# '''
# Custom training loop in order to avoid reading in entire training or testing dataset in RAM.
# '''
# BATCH_SIZE = 32
# def load_data(image_folder_path, label_path):  
#     '''
#     Loading training and testing databases. 
#     '''
    
#     # Reading in labels and creating a dictionary
#     reader = csv.reader(open(label_path, 'r')) # Reads in header as well but thats not important. 
#     label_dict = dict([(key, value) for key, value in reader])
        
#     image_paths = []
#     image_labels = []
#     for name in os.listdir(image_folder_path):    
#         image_path = os.path.join(image_folder_path, name) 
#         image_paths.append(image_path)
        
#         no_extension = name[:name.index('.')]
#         image_labels.append(int(label_dict[no_extension]))
    
#     image_paths_ds = tf.data.Dataset.from_tensor_slices(image_paths)
#     labels_ds = tf.data.Dataset.from_tensor_slices(image_labels)
#     ds = tf.data.Dataset.zip((image_paths_ds, labels_ds))
       
#     # Now determining how data is fed.
#     ds = ds.shuffle(buffer_size=len(image_paths)).repeat().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

#     return ds, len(image_paths)

# train_image_path_ds,_ = load_data('/kaggle/input/preprocessed/Preprocessed_Data', '/kaggle/input/aptos2019-blindness-detection/train.csv')
# print('Data Loaded.')

In [ ]:
# from tensorflow import contrib
# tfe = contrib.eager 

# NUM_EPOCHES = 300
# SAVE_EVERY = 5 # Epoches.
# SAVE_VERSIONS = 5
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
# global_step = tf.Variable(0) # Stores the number of learning steps taken.

# def loss(inputs, labels):
#     logits = model(inputs)
#     return tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

# def grad(inputs, labels):
#     with tf.GradientTape() as tape:
#         loss_value = loss(inputs, labels)
#     return loss_value, tape.gradient(loss_value, model.trainable_variables)

# def get_image(image_path):
#     image_file = tf.read_file(image_path)
#     image = tf.image.decode_png(image_file, channels=3)
#     image = tf.cast(image/255,tf.float32)
#     return image

# save_version = 0
# for epoch in range(NUM_EPOCHES):
#     epoch_loss_avg = tfe.metrics.Mean()
#     epoch_accuracy = tfe.metrics.Accuracy()

#     # Training loop where one loop contains a mini-batch of elements.
#     for image_paths, labels in train_image_path_ds:
#         # Obtains image from path. 
#         images = np.array([])
#         image_paths = image_paths.numpy()
#         for image_path in image_paths:
#             images = np.append(images, get_image(image_path))
#         # Optimize the model
#         loss_value, grads = grad(images, labels)
#         optimizer.apply_gradients(zip(grads, model.trainable_variables), global_step)

#         # Track progress
#         epoch_loss_avg(loss_value)  # add current batch loss
#         # compare predicted label to actual label
#         epoch_accuracy(model(image), y)

#     # end epoch
#     train_loss_results.append(epoch_loss_avg.result())
#     train_accuracy_results.append(epoch_accuracy.result())

#     print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,epoch_loss_avg.result(),epoch_accuracy.result()))
    
#     if epoch + 1 % SAVE_EVERY == 0:
#         save_version = save_version % SAVE_VERSIONS
#         model.save_weights('model_weights' + str(save_version) + '.h5', save_format='h5')
        
# '''
# Still need to create new model that excludes optimizers. 
# '''